<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?" data-toc-modified-id="Признаки-умножают-на-обратимую-матрицу.-Изменится-ли-качество-линейной-регрессии?-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

## Загрузка данных

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.isnull().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [5]:
features = data.drop(columns=['Страховые выплаты'])
target = data['Страховые выплаты']

## Умножение матриц

### Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

При подстановке в формулы вместо X - XP, получаем:

Предсказания:

$$
a = XPw
$$

Формула обучения:

$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$

Тогда формула предсказания:
$$
a = XP((XP)^T XP)^{-1} (XP)^T y
$$

При раскрытии скобок для транспонируемых матриц меняется порядок умножения:

$$
a = XP(P^T X^T XP)^{-1} P^T X^T y
$$

Далее раскрываем скобки для инверсии, порядок снова меняется:

$$
a = XP(P)^{-1}(P^T X^T X)^{-1} P^T X^T y
$$

$$
a = XP(P)^{-1}(X^T X)^{-1} P^{T^{-1}} P^T X^T y
$$

Произведение матрицы на обратную даёт единичную матрицу:

$$
a = XE(X^T X)^{-1} E X^T y
$$

И умножение на единичную матрицу оставляет исходную матрицу, получаем:

$$
a = X(X^T X)^{-1} X^T y
$$

что эквивалентно формуле предсказаний, для исходного набора признаков:

$$
a = Xw
$$

## Алгоритм преобразования

**Алгоритм**

Защита данных должна подразумевать создание обратимой матрицы, шифрующей данные, и обратной ей матрицы для расшифровки.

Для этого можно создать объект, который будет:

- принимать на вход признаки и возвращать обработанную матрицу
- возвращать оригинальную матрицу при подаче обработанной. 

При подаче на вход необработанных признаков, будет случайным образом генерироваться обратимая матрица нужного размера, и обратная ей, для последующей расшифровки. матрицы должны быть спрятаны внутри объекта

**Обоснование**


Из ответа на предыдущий вопрос, мы знаем, что при обучении модели линейной регрессии, и последующем предсказании, влияние матрицы, на которую умножаются признаки, сходит на нет. 

Но для этого признаки, для которых предсказываются значения, должны быть умножены на ту же самую матрицу, на которую умножались признаки для обучения модели.

## Проверка алгоритма

Создаём объект с необходимым функционалом

In [6]:
class InverseEncoder:
    #фиксация рандома при инициализации
    def __init__(self, random_state=None):
        self.__rstate = random_state
    
    def fit(self, X):
        self.__inverse(X)
        return self
    
    def __inverse(self, X):
        np.random.seed(self.__rstate)
        self.__P = np.random.randint(1000, size=(X.shape[1], X.shape[1]))
        
        try:
            self.__P_inv = np.linalg.inv(self.__P)
            
        except np.linalg.LinAlgError:
            self.__inverse(X)
            
    def transform(self, X):
        return np.dot(X, self.__P)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    
    def decode(self, X):
        reversed = np.dot(X, self.__P_inv)
        
        return np.dot(X, self.__P_inv)

Инициализируем объект и проверяем функционал

In [7]:
Encoder = InverseEncoder()
X = features

Encoder.fit(X)
XP = Encoder.transform(X)
XP

array([[  315346., 14169537., 19222183., 31622928.],
       [  246794., 10867132., 14735187., 24236180.],
       [  137774.,  6008345.,  8145212., 13395879.],
       ...,
       [  211756.,  9677804., 13132458., 21607788.],
       [  206468.,  9337946., 12670549., 20845727.],
       [  256068., 11594072., 15731019., 25881465.]])

In [8]:
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


Не видно ничего общего с исходными данными

In [9]:
Encoder.decode(XP)

array([[ 1.00000000e+00,  4.10000000e+01,  4.96000000e+04,
         1.00000000e+00],
       [ 0.00000000e+00,  4.60000000e+01,  3.80000000e+04,
         1.00000000e+00],
       [-1.81898940e-12,  2.90000000e+01,  2.10000000e+04,
         1.45519152e-11],
       ...,
       [ 0.00000000e+00,  2.00000000e+01,  3.39000000e+04,
         2.00000000e+00],
       [ 1.00000000e+00,  2.20000000e+01,  3.27000000e+04,
         3.00000000e+00],
       [ 1.00000000e+00,  2.80000000e+01,  4.06000000e+04,
         1.00000000e+00]])

На первый взгляд, на исходные данные не похоже. Но, если обратная матрица дробная, то могла сказаться погрешность при операциях с плавающей точкой, округлим результат

In [10]:
pd.DataFrame(Encoder.decode(XP)).round(2)

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,0.0,46.0,38000.0,1.0
2,-0.0,29.0,21000.0,0.0
3,-0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,0.0
...,...,...,...,...
4995,0.0,28.0,35700.0,2.0
4996,-0.0,34.0,52400.0,1.0
4997,0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0


Операция сравнения покажет ложный результат, но по факту, вернулись к прежним значениям

Теперь проверим результаты линейной регрессии

In [11]:
class LinReg:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(np.dot(X.T, X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0


In [12]:
model = LinReg()
model.fit(X, target)
predictions = model.predict(X)
print(r2_score(target, predictions))

0.4249455028666801


In [13]:
model = LinReg()
model.fit(XP, target)
predictions = model.predict(XP)
print(r2_score(target, predictions))

0.4249455028644853


Значения практически одинаковые, некоторая разница объясняется погрешностью операций с плавающей точкой

Дополнительно проверим на модели из sklearn

In [14]:
model2 = LinearRegression(fit_intercept=True, normalize = True)
model2.fit(X, target)
predictions = model2.predict(X)
print(r2_score(target, predictions))

0.4249455028666801


In [15]:
model2 = LinearRegression(fit_intercept=True, normalize = True)
model2.fit(XP, target)
predictions = model2.predict(XP)
print(r2_score(target, predictions))

0.4249455028666783


## Вывод

Было подтверждено предположение о том, что умножение матрицы признаков на обратимую матрицу, преобразит её до неузнаваемости и при этом не повлечёт за собой ухудшения качества линейной регрессии.

Реализован алгоритм такого умножения, со случайной генерацией и скрытым хранением обратной матрицы, что обеспечивает некоторую безопасность процедуры.